# Machine Translation using Transformers

Language translation is one of the most important task in NLP world. As people get connected with each other, language translation helps them to understand each other better. 

Machine translation can formally be identified as the process of reproducing human translation using machine transductions(Transduction means taking structures we perceive and representing them with another form).

And in real life, this is a complicated task due to many reasons. For example lets say there's a sentence A which is written in some language. Then we are required to translate this sentence in to another language. A human translator comes and translate the sentence A to give an translated output. But would another human translator give a same translation to the sentence A? This is probably not! But the problem is that both translated sentences would have same meaning. So when we try to build a model to do translation, our model should be able to understand/perform the same.

To see how such task is achieved we will look at a task of tranlating French/English. We will be using the WMT(Workshop of Machine Translation) dataset provided in [this url](https://www.statmt.org/europarl/v7/fr-en.tgz).
